# Classical Segmentation
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Connected component labelling](#connectedcomp)
* [Watershed and random-walk segmentation](#watershed)
* [Spot detector for segmentation of nuclei](#spotdetect)


<a id="intro"></a> 
## Introduction
One of the essential problems in bioimage analysis is instance segmentation or partitoning of the image into individual objects such as cells, nuclei, filaments, organelles, etc. This step is crucial since we are interested in characterizing the morphology and quantifying key phenotypic parameters of individual objects. In this notebook we will work on nucleus segmentation in leukemia cells and will explore the classical segmentation approaches that do not rely on machine learning

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

We will start by loading the image stack, applying maximum intensity projection (MIP) and thresholding the image using Otsu method:

In [ ]:
from base.utils import load_imgstack
imgstack = load_imgstack(fname="data/BiTE/Tag2-r04c02f1.tiff")
mip=np.max(imgstack, axis=0)

In [ ]:
from transform.process import threshold_img
from base.plot import plot_channels

hoechst = mip[:,:,2]**0.4
# threshold the image of nuclei
img_th = threshold_img(hoechst, method='otsu', binary=True)

The thresholded image is binarized (`binary=True`):

In [ ]:
plot_channels([hoechst, img_th],nrow=1, ncol=2,
             titles=['Image of nuclei', 'Thresholded image'],
             cmap='gray')

<a id="connectedcomp"></a> 
## Connected component labelling
Thresholding  has already separated most of the foreground pixels from the dark background. We can use connected component labelling to partition this binarized image of nuclei into islands of connected foreground pixels:

In [ ]:
from skimage.measure import label
from skimage.color import label2rgb

In [ ]:
segm = label(img_th, connectivity=1)

The `label` function performs connected component labelling (with a 4-neighbor scheme by default). It returns an array with the same shape as the original image but with every pixel labelled as belonging to one of the connected components:

In [ ]:
segm

In [ ]:
print("Found %d unique labels (connected components)" % (len(np.unique(segm)) - 1))

We can use `label2rgb` function to overlay the identified connected components  with the original  image of the nuclei:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(label2rgb(segm, image=hoechst, bg_label=0))
ax.axis('off')

Here the color indicates pixel label, i.e. which connected component the pixel belongs to. In many cases individual nuclei are identified ("segmented") using this simple heuristic. There are however a number of grape-shaped clusters of merged nuclei which could not be separated from one another and which form a single connected region. Note that the colors are cycled, i.e. repeated in some cases for regions with different labels. 


Once an image is labeled, we can use `regionprops` function to extract various region features for the individual labelled components:

In [ ]:
from skimage.measure import regionprops

In [ ]:
feats =  regionprops(label_image=segm, intensity_image=hoechst)

In [ ]:
len(feats)

For each of the 780 connected components there is a number of region properties that were computed with the call of `regionprops`. The keys of the features are:

In [ ]:
feat_keys = [f for f in feats[0]]
print(feat_keys)

We can extract some of these features and plot their distributions:

In [ ]:
def get_feattable(feats, keys):
    return pd.DataFrame({key: [f[key] for f in feats] for key in keys})

In [ ]:
feat_df = get_feattable(feats, keys=['area', 'eccentricity', 'mean_intensity', 'perimeter'])

In [ ]:
feat_df.head()

In [ ]:
sn.pairplot(feat_df)

## Watershed and random-walk segmentation
<a id="watershed"></a> 

## Spot detector for segmentation of nuclei
<a id="spotdetect"></a> 

In [ ]:
from transform.process import threshold_img
from skimage.feature import blob_log

In [ ]:
img_th = threshold_img(hoechst, method='otsu')
blobs = blob_log(img_th,
                 min_sigma=10, max_sigma=12, threshold=0.05)

A useful transformation is `shape_index` which is a measure of local curvature of the intensity landscape at every pixel. In the intensity landscape bright regions are ridges and hills, while the image background is a flat planar surface. Shape index maps every pixel value to the $[-1,1]$ range, with concave landscape pixels becoming negative, while convex regions (e.g. bright spots) are mapped to positive values.

Thus hape index will enhance the appearance of the bright spots. Image background (flat intensity landscape) will get `NaN` values after shape index is applied.

In [ ]:
from skimage.feature import shape_index
from segment.cv_methods import nantonum

In [ ]:
img_s = shape_index(img_th)
print("Number of NaN pixels: %d" % np.sum(np.isnan(img_s)))

Most of the `NaN`-valued pixels are most likely background pixels. We can convert `NaN` values to -1 using `nantonum` function and detect blobs in this enhanced image.

In [ ]:
img_enh = nantonum(img_s, pad=-1)

First visualize:

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img_enh, cmap='gray')

In [ ]:
# run LoG blob detection on the shape-index enhanced image
blobs_enh = blob_log(img_enh,
                 min_sigma=9, max_sigma=11, threshold=0.05)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
for blob in blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r-2, color='yellow', linewidth=1.4, fill=False)
    ax.add_patch(c)
for blob in blobs_enh:
    y, x, r = blob
    c = plt.Circle((x, y), r+2, color='magenta', linewidth=1.4, fill=False)
    ax.add_patch(c)
ax.imshow(hoechst, cmap='gray')
ax.axis('off')

We can see that some of the low-intensity spots are captured now if we use `blob_log` on an enhanced image because shape index only transforms the image based on local curvature of the landscape (weak spots are also "hills")